In [283]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import concurrent
from time import sleep
from IPython.display import display, Math, Latex

O que este notebook ira realizar:
-------------------------------

1. Acessar o site da olx utilizando a biblioteca **BeautifulSoup**, ecnontrar a lista de preços e salvar cada item em uma lista de "Tags"
2. Encontrar os valores (municipio, estado , categoria, titulo, preco, link) de cada anuncio na lista
3. Salvar em um DataFrame do Pandas
4. Acessar cada um dos links de cada anuncio no DataFrame, no campo **href** e retira dele o CEP de cada anuncio criando um dicionario neste formato {"href":"cep"} **(Esta parte do codigo precisa ser aprimorada atualmente demora em torno de 5 à 7min para executar)**
5. Adicionar o dicionario de CEP ao dataframe de anuncios utilizando a função _.map_ do Pandas 
6. Salvar o DataFrame resultante em uma planilha Excel

In [284]:
labels = ['local', 'categoria', 'titulo', 'preco', 'href']
cep_dict = {}
url_leste = "http://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis?o="
url_oeste = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-oeste/imoveis?o='
url_centro = 'http://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis?o='
url_norte = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-norte/imoveis?o='
url_sul = 'http://sp.olx.com.br/sao-paulo-e-regiao/zona-sul/imoveis?o='

In [285]:
def retornaSoupOLX(url, num):
    li_list = []
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    for iter in range(num):
        sleep(1)
        r = requests.get(str(url)+str(iter), headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        for ultag in soup.find_all('ul', {'class': 'list', 'id':'main-ad-list' }):
            for litag in ultag.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['item']):
                li_list.append(litag)
    return li_list

In [286]:
def tiraEspecialString(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '')

In [287]:
def tiraEspecialFloat(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '').replace(',', '.')

In [288]:
%%time

def criaListaValores(li_list):
    valores = []
    for elements in li_list:
        x = BeautifulSoup(str(elements), 'html.parser')
        #encontra a parte onde estão as informaçoes no html
        info = x.find('div', {'class':'col-2'})
        #encontra a string onde estão o municipio e estado separados por virgula
        cidade_estado = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-region'}))
        #encontra a categoria do anuncio
        categoria = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-category'}))
        #encontra o titulo do anuncio
        titulo = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-1 mb5px'}).find('h3', {'class':'OLXad-list-title mb5px'}))
        #encontra o html do preço
        preco = x.find('div', {'class':'col-3'})
        
        #trata o html do preço verificando se existe ou não
        if preco.find('p', {'class':'OLXad-list-price'}):
            preco = tiraEspecialFloat(x.find('div', {'class':'col-3'}).find('p', {'class':'OLXad-list-price'}))
        else:
            preco = 0.0
        #retorna a tag href "link do anuncio"
        href=x.find('a', {'class':'OLXad-list-link'})
        
        #retorna o conteudo do atributo href na tag href
        href = href['href']
        
        valores.append([cidade_estado, categoria, titulo, preco, href])
    return valores
        

Wall time: 0 ns


In [289]:
def findCEP(url):
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    regx = re.compile('\d{5}-\d{3}')
    r = requests.get(url, headers = headers)
    sleep(1)
    soup = BeautifulSoup(r.content, 'html.parser')
    cep = soup.find(lambda tag: tag.name == 'strong' and tag.get('class') == ['description'] and regx.match(tag.text.strip(' \t\n\r')) is not None)
    if cep:
        cep_dict.update({url:cep.text.strip(' \t\n\r')})
    else:
        cep_dict.update({url:''})

In [290]:
%%time
#cria as Soups de cada região
soup_leste = retornaSoupOLX(url_leste, 1)
soup_sul = retornaSoupOLX(url_sul,1)
soup_norte = retornaSoupOLX(url_norte, 1)
soup_oeste = retornaSoupOLX(url_oeste, 1)
soup_centro = retornaSoupOLX(url_centro, 1)

Wall time: 8.45 s


In [291]:
%%time
#cria lista com os valores separados de cada região
valores_leste = criaListaValores(soup_leste)
valores_sul = criaListaValores(soup_sul)
valores_norte = criaListaValores(soup_norte)
valores_oeste = criaListaValores(soup_oeste)
valores_centro = criaListaValores(soup_centro)

Wall time: 1.3 s


In [292]:
# valores = criaListaValores(soup_leste)

In [293]:
%%time
#cria o dataframe de cada uma das regiões
df_norte = pd.DataFrame(valores_norte, columns=labels)
df_sul = pd.DataFrame(valores_sul, columns=labels)
df_leste = pd.DataFrame(valores_leste, columns=labels)
df_oeste = pd.DataFrame(valores_oeste, columns=labels)
df_centro = pd.DataFrame(valores_centro, columns=labels)

Wall time: 5.5 ms


In [294]:
#cria uma coluna com a Zona de cada dataframe
df_norte['zona'] = 'Norte'
df_sul['zona'] = 'Sul'
df_leste['zona'] = 'Leste'
df_oeste['zona'] = 'Oeste'
df_centro['zona'] = 'Centro'

In [295]:
#criar uma lista de DataFrames e juntalos em apenas um
df = pd.DataFrame()
lista_df_zona = [
    df_norte,
    df_sul,
    df_leste,
    df_oeste,
    df_centro
]
df = df.append(lista_df_zona, ignore_index=True)

In [296]:
df['categoria'] = df['categoria'].str.replace('- Anúncio Profissional', 'Profissional')
df['categoria'] = df['categoria'].str.replace('Profissional', '- Profissional')
df['categoria'], df['profissional'] = df['categoria'].str.split('-', 1).str
df['profissional']=df.profissional.replace(' Profissional', 'S')
df['profissional']=df.profissional.fillna('N')
df.head()

,local,categoria,titulo,preco,href,zona,profissional
0,"São Paulo, Carandiru",Indústria e comércio,Galpão/Depósito/Armazém,18000,http://sp.olx.com.br/sao-paulo-e-regiao/indust...,Norte,S
1,"São Paulo, Santana",Apartamentos,Apartamento Padrão,480000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S
2,"São Paulo, Jardim Dos Francos",Casas,Casa renda oportunidade,250000,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,N
3,"São Paulo, Jardim Tiro Ao Pombo",Apartamentos,Apto COD L589,850,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S
4,"São Paulo, Vila Nilo",Casas,Sobrado COD L586,1200,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S


In [297]:
#selecionar casas com preços acima de 100000
df_casa = df[df['categoria'] == 'Casas']
df_casa['preco'] = df_casa['preco'].astype(float)
df_casa = df_casa[df_casa['preco']!=0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [298]:
list_href = list(df_casa['href'])

In [299]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as e:
     e.map(findCEP, list_href)

Wall time: 5.5 s


In [300]:
df_casa['cep'] = df_casa['href'].map(cep_dict)

In [301]:
#Aqui dropa o antigo local e separa o 'São Paulo, Bairro' em duas colunas
df_casa = pd.concat([df_casa , df_casa['local'].str.split(',', n=1, expand=True)], axis =1)
df_casa.drop(['local', 0], axis=1, inplace=True)

In [303]:
df_casa.columns = ['categoria', 'titulo', 'preco', 'href','zona','profissional', 'cep', 'bairro']

In [304]:
df_casa.head()

,categoria,titulo,preco,href,zona,profissional,cep,bairro
2,Casas,Casa renda oportunidade,250000.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,N,02874-140,Jardim Dos Francos
4,Casas,Sobrado COD L586,1200.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S,02278-040,Vila Nilo
5,Casas,Sobrado em condomínio na Vila Pauliceia com 2 ...,1500.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,N,02302-100,Vila Paulicéia
6,Casas,Casa COD L600,750.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S,02845-080,Brasilândia
11,Casas,Ref: 9823 - ASSOBRADADA em JARAGUA para venda,440000.0,http://sp.olx.com.br/sao-paulo-e-regiao/imovei...,Norte,S,02995-150,Jardim São João


In [305]:
df.loc[:,df.columns != 'cep'].to_excel('../1-OLX_WebScrap/1_OLX_SEM_CEP.xlsx')
df.to_excel('../1-OLX_WebScrap/1_OLX_COM_CEP.xlsx')